In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
import ast

In [ ]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", device_map = 'mps')
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")


In [ ]:
path = '../self_contra/wino_bias_question_similarity/'

In [ ]:
files = os.listdir(path)
if '.DS_Store' in files:
    files.remove('.DS_Store')
len(files), files

In [ ]:
files.remove('pro_stereotyped_type1.txt.dev.csv')
files.remove('anti_stereotyped_type2.txt.dev.csv')
files.remove('anti_stereotyped_type1.txt.dev.csv')
files.remove('anti_stereotyped_type1.txt.test.csv')
files.remove('pro_stereotyped_type2.txt.test.csv')
files.remove('pro_stereotyped_type2.txt.dev.csv')
files.remove('anti_stereotyped_type2.txt.test.csv')


In [ ]:
files=['anti_stereotyped_type2.txt.dev.csv']


In [ ]:

def get_mistral_pred(question): 
    messages = [
        {
            "role":"user",
            "content":f"""Question: {question} 
            Give your reason first, then answer, answer should be in maximum 3 to 4 words. 
            Follow the format: reason:[reason]:\nAnswer:[Answer]\n"""
        }
    ]


    model_inputs = tokenizer.apply_chat_template(messages, return_tensors = "pt").to('mps')
    pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id
    attention_mask = model_inputs.ne(pad_token_id).long()

    generated_ids = model.generate(model_inputs, max_new_tokens = 1000, do_sample = True, 
                                   attention_mask=attention_mask, pad_token_id=pad_token_id)
    output = tokenizer.batch_decode(generated_ids, skip_special_tokens = True)[0]
    #print(output)
    if 'Answer:[Answer]' in output:
        index = output.index('Answer:[Answer]')
        a = output[index+25:]
        b = a.index('Answer')
        
        reason = a[:b]
        result = a[b+8:]
        
        return reason, result
    return None, None
        

In [ ]:
for num, file_name in enumerate(files):    
    pred = []
    reasons = []
    
    real_questions = []
    real_ground_truths = []
    real_reasonings = []
    real_predictions = [] 
    real_generated_questionss = []
    real_similarity_values = []
    real_question_nums = []
    df = pd.DataFrame(columns = ['question', 'ground_truth', 'reasoning', 'prediction', 'generated_questions', 'similarity_value', 'question_num', 'reason_for_generated', 'pred_for_generated'])

    data = pd.read_csv(path + file_name)
    for i, row in data.iterrows(): 
        if i < 363:
            continue
        print(num, file_name, i)
        real_question = row['question']
        real_ground_truth = row['ground_truth']
        real_reasoning = row['reasoning']
        real_prediction = row['prediction']
        real_generated_questions = row['generated_questions']
        real_similarity_value = row['similarity_value']
        real_question_num = row['question_num']
        
        line = str(row['question'])
        question = line.split('.')[0]
        question_num = row['question_num']
        generated_questions = row['generated_questions']
        generated_questions = ast.literal_eval(generated_questions)
        if question_num == 0:
            q = generated_questions['question1']
            question += q
            reason, result = get_mistral_pred(question)
        elif question_num == 1:
            q = generated_questions['question2']
            question += q
            reason, result = get_mistral_pred(question)
        elif question_num == 2:
            q = generated_questions['question3']
            question += q
            reason, result = get_mistral_pred(question)
        else:
            reason, result = None, None
        
        
        reason, result = get_mistral_pred(question)
        reasons.append(reason)
        pred.append(result)
        
        real_questions.append(real_question) 
        real_ground_truths.append(real_ground_truth)
        real_reasonings.append(real_reasoning) 
        real_predictions.append(real_prediction) 
        real_generated_questionss.append(generated_questions)
        real_similarity_values.append(real_similarity_value)
        real_question_nums.append(real_question_num)
        
        
    
    df['reason_for_generated'] = reasons
    df['pred_for_generated'] = pred
    
    df['question'] = real_questions
    df['ground_truth'] = real_ground_truths
    df['reasoning'] = real_reasonings
    df['prediction'] = real_predictions
    df['generated_questions'] = real_generated_questionss
    df['similarity_value'] = real_similarity_values
    df['question_num'] = real_question_nums
    df.to_csv('../self_contra/wino_bias_reason_answer_for_generated_question_by_mistral/' + file_name + '3.csv')
    

In [ ]:
df['reason_for_generated'] = reasons
df['pred_for_generated'] = pred

df['question'] = real_questions
df['ground_truth'] = real_ground_truths
df['reasoning'] = real_reasonings
df['prediction'] = real_predictions
df['generated_questions'] = real_generated_questionss
df['similarity_value'] = real_similarity_values
df['question_num'] = real_question_nums
df.to_csv('../self_contra/wino_bias_reason_answer_for_generated_question_by_mistral/' + file_name + '2.csv')

